In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = '../scrape_func/'

In [3]:
games = pd.read_csv(data_path + 'data/game_list.csv')
horoyois = pd.read_csv(data_path + 'data/horoyoi_list.csv')

games.head()
# horoyoi.head()

,name,img,link
0,\nカルカソンヌ(Carcassonne)\n,https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=c...
1,\n世界の七不思議(7Wonders)\n,https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...
2,\nキングドミノ(Kingdomino)\n,https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=k...
3,\nニムト(6nimmt!)\n,https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...
4,\nテラミスティカ(TerraMystica)\n,https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=t...


In [4]:
# 名前修正

games['name'] = games['name'].apply(lambda x: x.replace('\n', ''))

games.head()

,name,img,link
0,カルカソンヌ(Carcassonne),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=c...
1,世界の七不思議(7Wonders),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...
2,キングドミノ(Kingdomino),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=k...
3,ニムト(6nimmt!),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...
4,テラミスティカ(TerraMystica),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=t...


In [5]:
horoyois.head()

,name,img
0,白いサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...
1,グレープ,https://www.suntory.co.jp/rtd/horoyoi/assets/i...
2,もも,https://www.suntory.co.jp/rtd/horoyoi/assets/i...
3,アイスティーサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...
4,白ぶどう,https://www.suntory.co.jp/rtd/horoyoi/assets/i...


In [6]:
import numpy as np
import cv2
import sklearn
from sklearn.cluster import KMeans

# メインの関数
def get_main_color(img_path):
    """ 
    ex: img_path = 'images/horoyoi/グレープ.png' 
    代表色の[r, g, b]を返します。
    """

    def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
        try:
            n = np.fromfile(filename, dtype)
            img = cv2.imdecode(n, flags)
            return img
        except Exception as e:
            print(e)
            return None
        
    def toRGB(color_list):
        return [color_list[2], color_list[1], color_list[0]]
        
    _sample_image = imread(data_path + img_path)

    # print('shape:' + str(_sample_image.shape))

    # to 2dimention
    sample_image = _sample_image.reshape(
        (_sample_image.shape[0] * _sample_image.shape[1], 3))

    # print('converted shape:' + str(sample_image.shape))
    
    # k-means
    cluster = KMeans(n_clusters=5)
    cluster.fit(X=sample_image)
    
    result = cluster.fit_predict(sample_image)
    
    center = cluster.cluster_centers_
    _, counts = np.unique(result, return_counts=True)
    
    # 代表色だけを取得する

    max_idx = np.argmax(counts)
    main_bgr_color = center[max_idx]
    
    return toRGB(main_bgr_color)

In [12]:
# 例
game_names = games['name']
path = 'images/game/{}.png'.format(game_names[3])
get_main_color(path)

[240.69328351722865, 196.12937700930843, 7.840298896508806]

In [16]:
def make_csv_with_color(df, isgame):
    if isgame:
        path='images/game/'
    else:
        path = 'images/horoyoi/'
    
    names = df['name']
    
    color_list = []
    
    for name in names:
        file_name = '{}.png'.format(name)
        file_path = path + file_name
        
        try:
            color = get_main_color(file_path)
        except:
            color = None
            print('error in {}'.format(name))
        
        color_list.append(color)
        
    df['color'] = color_list
        

In [14]:
make_csv_with_color(horoyois, isgame=False)
make_csv_with_color(games, isgame=True)

horoyois.head()

[Errno 2] No such file or directory: '../scrape_func/images/horoyoi白いサワー.png'
error in 白いサワー
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiグレープ.png'
error in グレープ
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiもも.png'
error in もも
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiアイスティーサワー.png'
error in アイスティーサワー
[Errno 2] No such file or directory: '../scrape_func/images/horoyoi白ぶどう.png'
error in 白ぶどう
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiハピクルサワー.png'
error in ハピクルサワー
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiミックスフルーツ.png'
error in ミックスフルーツ
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiカシスとオレンジ.png'
error in カシスとオレンジ
[Errno 2] No such file or directory: '../scrape_func/images/horoyoi梅酒ソーダ.png'
error in 梅酒ソーダ
[Errno 2] No such file or directory: '../scrape_func/images/horoyoiグレフルソルティ.png'
error in グレフルソルティ
[Errno 2] No such file or directory: '../scrape_fu

,name,img,color
0,白いサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,None
1,グレープ,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,None
2,もも,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,None
3,アイスティーサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,None
4,白ぶどう,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,None


In [15]:
games.head()

,name,img,link,color
0,カルカソンヌ(Carcassonne),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=c...,"[2.2240218918997954, 1.5503291176551102, 1.716..."
1,世界の七不思議(7Wonders),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...,"[3.8158094115787833, 5.181244212494974, 6.7913..."
2,キングドミノ(Kingdomino),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=k...,"[0.9383290266961382, 2.6864771748668517, 1.176..."
3,ニムト(6nimmt!),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=s...,"[240.69334376084328, 196.1283454987948, 7.8330..."
4,テラミスティカ(TerraMystica),https://x.boardgamearena.net/data/themerelease...,https://ja.boardgamearena.com/gamepanel?game=t...,"[0.8769417930027643, 0.9429159648026229, 0.777..."


In [19]:
make_csv_with_color(horoyois, isgame=False)

horoyois.head()

,name,img,color
0,白いサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,"[241.6324476819914, 241.15002736951718, 239.27..."
1,グレープ,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,"[236.8805715924142, 189.94545845730283, 214.68..."
2,もも,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,"[225.60381794246314, 188.2640586796929, 177.75..."
3,アイスティーサワー,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,"[145.45541578255938, 36.924269980834424, 36.19..."
4,白ぶどう,https://www.suntory.co.jp/rtd/horoyoi/assets/i...,"[209.11797265989802, 223.1601322793002, 121.24..."


In [22]:
games.to_csv(data_path + 'data/add_color/game_list.csv')
horoyois.to_csv(data_path + 'data/add_color/horoyoi_list.csv')

In [32]:
# 読み込み例
horoyois = pd.read_csv(data_path + 'data/add_color/horoyoi_list.csv')

[float(x) for x in horoyois['color'][0].strip('[]').replace(',', '').split()]

[241.6324476819914, 241.15002736951718, 239.27397644807957]